In [ ]:
!git clone https://github.com/yhenon/pytorch-retinanet.git

fatal: destination path 'pytorch-retinanet' already exists and is not an empty directory.


In [ ]:
!cp -r "/content/pytorch-retinanet" "/content/drive/My Drive"

In [ ]:
%cd "/content/drive/My Drive/pytorch-retinanet"
import re
import cv2
import time
import numpy as np
import pandas as pd
import os

from skimage import io




import torch
import torch.optim as optim
import torchvision.transforms as T
from torchvision.utils import make_grid 
from torch.utils.data import DataLoader, Dataset

from retinanet import model
from retinanet.dataloader import collater, Resizer, Augmenter, Normalizer, UnNormalizer

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

DIR_TRAIN = "/content/drive/My Drive/df_fold_0/images/train_image"
DIR_VAL = "/content/drive/My Drive/df_fold_0/images/val_image"

/content/drive/My Drive/pytorch-retinanet


In [ ]:
import PIL
from PIL import Image
import skimage.io
import os

In [ ]:
def load_image(image_index, dir):
      #print(image_index)
      image_name = os.path.join(dir, image_index)
      img = skimage.io.imread(image_name)

      if len(img.shape) == 2:
          img = skimage.color.gray2rgb(img)

      return img.astype(np.float32)/255.0

In [ ]:
class DhakaAi(Dataset):
    def __init__(self, dataframe, image_dir, mode = "train", transforms = None):
        
        super().__init__()
        self.image_ids = dataframe['image_id']
        self.df = dataframe
        self.image_dir = image_dir
        self.mode = mode
        self.transforms = transforms
    

    def __getitem__(self, index: int):

        
        # Retriving image id and records from df
        #records = self.df[self.df['image_id'] == image_id]

        # Loading Image
        
        image_id = self.image_ids[index]
        image = load_image(image_id, self.image_dir)
        image_shape = image.shape
        
        # If mode is set to train, then only we create targets
        if self.mode == "train" or self.mode == "valid":
          label_dir = self.image_dir.replace("images", "labels")
          label_dir = label_dir + "/" + image_id.replace("jpg", "txt")
          label_file = open(label_dir, "r")
          boxes = []
          lines = label_file.read().split("\n")
          for line in lines[:-1]:
            box = np.zeros((5, 1))
            line = line.split(" ")
            box[4] = int(line[0])
            box[0] = int((float(line[1]) - float(line[3]))*image_shape[1])
            box[1] = int((float(line[2]) - float(line[4]))*image_shape[0])
            box[2] = int((float(line[1]) + float(line[3]))*image_shape[1])
            box[3] = int((float(line[2]) + float(line[4]))*image_shape[0])
            boxes.append(box)
          boxes = np.array(boxes, dtype=np.int64)
            
          # Applying Transforms
          sample = {'img': image, 'annot': boxes}
                
          if self.transforms:
            image = self.transforms(image)
            sample = {"img": image, "annot": boxes}

          return sample
        
        elif self.mode == "test":
            
          # We just need to apply transoforms and return image
          if self.transforms:
                
            
            image = self.transforms(image)
            sample = {'img' : image}
                
            return sample
      

    def __len__(self) -> int:
        return self.image_ids.shape[0]

    def collate_fn(batch):
        img, label, path, shapes = zip(*batch)  # transposed
        for i, l in enumerate(label):
            l[:, 0] = i  # add target image index for build_targets()
        return torch.stack(img, 0), torch.cat(label, 0), path, shapes
    

In [ ]:
transforms = T.Compose([Augmenter(),
                        Normalizer(),
                        Resizer(), 
                        T.ColorJitter(),
                        T.RandomHorizontalFlip(p=0.5), 
                        T.RandomPerspective(distortion_scale=0.5, p=0.5, interpolation=3, fill=0), 
                        T.RandomRotation(50, resample=False, expand=False, center=None, fill=None),
                        T.RandomVerticalFlip(p=0.5), 
                        T.RandomErasing(p=0.5, scale=(0.02, 0.33),ratio=(0.3, 3.3), value=0, inplace=False)])

In [ ]:
import os
import pandas as pd
train_list = os.listdir("/content/drive/My Drive/df_fold_0/images/train_image")
train_df = pd.DataFrame()
train_df["image_id"] = train_list
val_list = os.listdir("/content/drive/My Drive/df_fold_0/images/val_image")
val_df = pd.DataFrame()
val_df["image_id"] = val_list

In [ ]:
train_df.head()

,image_id
0,Pias (474).jpg
1,Pias (129).jpg
2,Navid_371.jpg
3,14.jpg
4,Numan_(409).jpg


In [ ]:
train_dataset = DhakaAi(train_df, DIR_TRAIN, mode = "train", transforms = transforms)
valid_dataset = DhakaAi(val_df, DIR_VAL, mode = "valid", transforms = transforms)

In [ ]:
# DataLoaders
train_data_loader = DataLoader(
    train_dataset,
    batch_size = 16,
    shuffle = True,
    num_workers = 4,
    collate_fn = collater
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size = 16,
    shuffle = True,
    num_workers = 4,
    collate_fn = collater
)


test_data_loader = DataLoader(
    valid_dataset,
    batch_size = 1,
    shuffle = True,
    num_workers = 4,
    collate_fn = collater
)

In [ ]:
### Utilize GPU if available

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.cuda.empty_cache()

In [ ]:
### I am using Pre-trained Resnet50 as backbone

retinanet = model.resnet50(num_classes = 2, pretrained = True)

# Loading Pre-trained model - if you load pre-trained model, comment above line.
#retinanet = torch.load("path_to_.pt_file")

In [ ]:
### Preparing model for training

# Defininig Optimizer
optimizer = torch.optim.Adam(retinanet.parameters(), lr = 0.0001)

# Learning Rate Scheduler
#lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma=0.5)

retinanet.to(device)

#No of epochs
epochs = 15

In [ ]:
def train_one_epoch(epoch_num, train_data_loader):
    
    print("Epoch - {} Started".format(epoch_num))
    st = time.time()
    
    retinanet.train()
    
    epoch_loss = []

    for iter_num, data in enumerate(train_data_loader):
                
        # Reseting gradients after each iter
        optimizer.zero_grad()
            
        # Forward
        classification_loss, regression_loss = retinanet([data['img'].cuda().float(), data['annot'].cuda().float()])
                
        # Calculating Loss
        classification_loss = classification_loss.mean()
        regression_loss = regression_loss.mean()

        loss = classification_loss + regression_loss

        if bool(loss == 0):
            continue
                
        # Calculating Gradients
        loss.backward()

        # Gradient Clipping
        torch.nn.utils.clip_grad_norm_(retinanet.parameters(), 0.1)
                
        # Updating Weights
        optimizer.step()

        #Epoch Loss
        epoch_loss.append(float(loss))

            
        print(
            'Epoch: {} | Iteration: {} | Classification loss: {:1.5f} | Regression loss: {:1.5f} | Running loss: {:1.5f}'.format(
                epoch_num, iter_num, float(classification_loss), float(regression_loss), np.mean(epoch_loss)))

        del classification_loss
        del regression_loss
        
    # Update the learning rate
    #if lr_scheduler is not None:
        #lr_scheduler.step()
        
    et = time.time()
    print("\n Total Time - {}\n".format(int(et - st)))

In [ ]:
def valid_one_epoch(epoch_num, valid_data_loader):
    
    print("Epoch - {} Started".format(epoch_num))
    st = time.time()
    
    epoch_loss = []

    for iter_num, data in enumerate(valid_data_loader):
                
        with torch.no_grad():
            
            # Forward
            classification_loss, regression_loss = retinanet([data['img'].cuda().float(), data['annot'].cuda().float()])

            # Calculating Loss
            classification_loss = classification_loss.mean()
            regression_loss = regression_loss.mean()
            loss = classification_loss + regression_loss

            #Epoch Loss
            epoch_loss.append(float(loss))

            print(
                'Epoch: {} | Iteration: {} | Classification loss: {:1.5f} | Regression loss: {:1.5f} | Running loss: {:1.5f}'.format(
                    epoch_num, iter_num, float(classification_loss), float(regression_loss), np.mean(epoch_loss)))

            del classification_loss
            del regression_loss
        
    et = time.time()
    print("\n Total Time - {}\n".format(int(et - st)))
    
    # Save Model after each epoch
    %cd "/content/drive/My Drive"
    torch.save(retinanet, "retinanet_gwd.pt")

In [ ]:
### Training Loop
for epoch in range(epochs):
    
    # Call train function
    train_one_epoch(epoch, train_data_loader)
    
    # Call valid function
    valid_one_epoch(epoch, valid_data_loader)

Epoch - 0 Started


IndexError: ignored